Momentum investing is investing in stocks that have increased in price the most. 

Here, we will select the 50 stocks with highest price momentum. Then, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks. (From project 1)

In [3]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats
import xlsxwriter
from datetime import datetime
import yfinance as yf

In [4]:
# pandas read html
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# clean data
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')

symbols_list = sp500['Symbol'].unique().tolist()

end_date = datetime.today().strftime('%Y-%m-%d')
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*6)

# stacking the data to make it easier to work with
df = yf.download(tickers=symbols_list, 
                 start=start_date, 
                 end=end_date).stack()

df

[*********************100%%**********************]  503 of 503 completed
C:\Users\Matthew\AppData\Local\Temp\ipykernel_5004\2987482327.py:15: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  end=end_date).stack()


Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2018-07-31 A        63.207775   66.040001   66.690002   64.540001   64.930000   
           AAL      38.702709   39.540001   39.869999   39.139999   39.259998   
           AAPL     45.223686   47.572498   48.035000   47.334999   47.575001   
           ABBV     70.257538   92.230003   92.629997   91.330002   91.650002   
           ABT      59.123009   65.540001   65.769997   65.260002   65.370003   
...                       ...         ...         ...         ...         ...   
2024-07-26 XYL     140.839996  140.839996  142.130005  137.820007  138.479996   
           YUM     128.050003  128.050003  129.039993  127.410004  127.690002   
           ZBH     111.290001  111.290001  112.279999  110.230003  110.790001   
           ZBRA    325.980011  325.980011  330.970001  323.000000  326.500000   
           ZTS     179.839996  179.839996  183.850006  179.259995  182.259995   

Price                   Volume  
Date       Ticker               
2018-07-31 A         2539200.0  
           AAL       5761100.0  
           AAPL    157492000.0  
           ABBV      8691800.0  
           ABT       5399400.0  
...                        ...  
2024-07-26 XYL       1074100.0  
           YUM       1874400.0  
           ZBH       1399400.0  
           ZBRA       458700.0  
           ZTS       2437300.0  

[746610 rows x 6 columns]

In [5]:
# we want to pull price in 1 yr stock return
tickers_list = df.index.get_level_values(1).unique()
AllYF = yf.Tickers(' '.join(tickers_list)).tickers
# this is a replacement for year1ChangePercent
# yeet = AllYF['AAPL'].info['52WeekChange']

In [14]:
df_columns = ['Ticker', 'Stock Price', 'One-Year Price Return', 'Number of Shares to Buy']

data_list = [
    [
        ticker,
        values.info.get('previousClose', None), 
        values.info.get('52WeekChange', None),
        'N/A'
    ]
    for ticker, values in AllYF.items()
]

final_df = pd.DataFrame(data=data_list, columns=df_columns)
# 2:02

In [16]:
# inplace will directly modify final_df
final_df.sort_values(by='One-Year Price Return', ascending=False, inplace=True)
# get the top 50 results (50 highest returns)
final_df = final_df[:50]
# reset the index to start from 0
final_df.reset_index(drop=True, inplace=True)

Ngl, this is really lame. We will now create a more realistic momentum strategy with 1, 3, 6, and 12 month returns.

- High-quality m-stocks show "slow and steady" outperformance over long periods of time.
- Low-quality m-stocks might not show any momentum for a long time, then surge upwards.
